In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/"My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


In [ ]:
data= pd.read_csv("Total_Features_5.csv")

In [ ]:
data.head()

,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,HTHG,HTAG,FTHG,FTAG,FTR
0,7,15,4,5,11,13,4,8,1,3,0,0,1,1,2,2,D
1,21,7,10,4,11,10,7,5,4,2,0,0,1,0,2,1,H
2,11,7,2,1,10,13,2,4,2,0,0,0,1,0,1,1,D
3,10,10,6,3,7,6,2,4,0,0,0,0,0,0,1,1,D
4,11,17,3,6,17,15,4,6,3,2,0,0,1,1,1,1,D


In [ ]:
data.shape

(289, 17)

In [ ]:
X = data.drop('FTR',axis=1)
y = data['FTR']
X.shape, y.shape

((289, 16), (289,))

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size = 0.2, random_state = 0, stratify=y)

Constant Feature Removal

In [ ]:
constant_filter = VarianceThreshold(threshold=0)
constant_filter.fit(X_train)

VarianceThreshold(threshold=0)

In [ ]:
constant_filter.get_support().sum()

16

In [ ]:
constant_list = [not temp for temp in constant_filter.get_support()]
constant_list

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [ ]:
X.columns[constant_list]

Index([], dtype='object')

In [ ]:
X_train_filter = constant_filter.transform(X_train)
X_test_filter = constant_filter.transform(X_test)

In [ ]:
X_train_filter.shape , X_test_filter.shape , X_train.shape

((231, 16), (58, 16), (231, 16))

Quasi Constant Feature Removal

In [ ]:
quasi_constant_filter = VarianceThreshold(threshold=0.01)

In [ ]:
quasi_constant_filter.fit(X_train_filter)

VarianceThreshold(threshold=0.01)

In [ ]:
quasi_constant_filter.get_support().sum()

16

In [ ]:
X_train_quasi_filter = quasi_constant_filter.transform(X_train_filter)
X_test_quasi_filter = quasi_constant_filter.transform(X_test_filter)

In [ ]:
X_train_quasi_filter.shape , X_test_quasi_filter.shape

((231, 16), (58, 16))

Remove Duplicate Features

In [ ]:
X_train_T = X_train_quasi_filter.T
X_test_T = X_test_quasi_filter.T

In [ ]:
type(X_train_T)

numpy.ndarray

In [ ]:
X_train_T = pd.DataFrame(X_train_T)
X_test_T = pd.DataFrame(X_test_T)

In [ ]:
X_train_T.shape , X_test_T.shape

((16, 231), (16, 58))

In [ ]:
X_train_T.duplicated().sum()

0

In [ ]:
duplicated_features = X_train_T.duplicated()
duplicated_features

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
dtype: bool

In [ ]:
features_to_keep = [not index for index in duplicated_features]

In [ ]:
X_train_unique = X_train_T[features_to_keep].T
X_test_unique = X_test_T[features_to_keep].T

In [ ]:
X_train_unique.shape , X_train.shape

((231, 16), (231, 16))

Building the ML Model

In [ ]:
def run_randomForest(X_train,X_test,y_train,y_test):
  clf = RandomForestClassifier(n_estimators=100,random_state=0,n_jobs=-1)
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  print('Accuracy on test set: ')
  print(accuracy_score(y_test, y_pred))

In [ ]:
%%time
run_randomForest(X_train_unique,X_test_unique,y_train,y_test)

Accuracy on test set: 
0.9655172413793104
CPU times: user 210 ms, sys: 29.9 ms, total: 240 ms
Wall time: 345 ms
